# Generar tablas

Correr esta notebook completa para generar las tablas que luego vamos a procesar en el análisis de regresión lineal en R

In [1]:
#
# Este boilerplate es para que Python encuentre el código 
#
%load_ext autoreload
%autoreload 2
%matplotlib inline

from nbhelpers import add_source_path

add_source_path("..")


In [2]:
%matplotlib inline

import pylab
import config
import logging
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from session import SessionMapper, load_session
from IPython.display import display
from pandasql import sqldf
from social import get_social_variables

social_variables = get_social_variables()

pysqldf = lambda q: sqldf(q, globals())

logger = logging.getLogger('main')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


sessions = [load_session(i) for i in range(1,13)]

Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_01/s01.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_02/s02.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_03/s03.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_04/s04.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_05/s05.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_06/s06.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_07/s07.objects.1.tasks not found. Building from scratch
Pickle for session /home/jmperez/projects/tesis/data/games-corpus/session_08/s08.objects.1

Generamos las tablas para cada feature:

In [3]:
import os
from analyzers import Analyzer


features = ["F0_MEAN", 
            "F0_MAX",
            "ENG_MAX", 
            "ENG_MEAN", 
            "SYLLABES_AVG", 
            'PHONEMES_AVG',
            'NOISE_TO_HARMONICS_RATIO',
            'SOUND_ALL_LOCAL_JITTER',
            'SOUND_ALL_LOCAL_SHIMMER',
            'SOUND_VOICED_LOCAL_JITTER',
            'SOUND_VOICED_LOCAL_SHIMMER'
]
all_features_analysis = pd.DataFrame(columns=features, dtype=float)

social_vars = [var for var in social_variables.columns if var not in ["session", "task", "speaker"]]

for feature in features:
    logger.info("Generating table for {}".format(feature))
    analysis = Analyzer(feature).analyze(sessions)
    joined_analysis = pysqldf("""
    SELECT * from analysis INNER JOIN social_variables ON 
        analysis.session=social_variables.session AND 
        analysis.task=social_variables.task AND
        analysis.speaker=social_variables.speaker""")
    
    table_path = os.path.join(config.TABLES_DIR, "%s.csv" % feature)
    joined_analysis.to_csv(table_path)
        

Generating table for F0_MEAN
There was an error calling /home/jmperez/projects/tesis/scripts/voice-analysis.praat with interval [528.4725, 528.945]
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
-?() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
uu-(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
?-() has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu(uu) has no defined phonemes. Defaulting to 0
uu-(uu) has no defined phonemes. Defaulting to 0
uu(u

In [4]:
mapper = SessionMapper(None)
for session in sessions:
    mapper.save(session)